In [ ]:
# 0.0 import packages

import numpy as np
import time

import torch
import torch.nn as nn
import torch.optim as optim
from numba import cuda as numba
from GPUtil import showUtilization as gpu_usage

import matplotlib.pyplot as plt

In [ ]:
# 0.2 GPU stuff

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("torch device: ", torch.cuda.get_device_name())
#device = torch.device("cpu")

# function to clear GPU memory
def free_gpu_cache():                        
    torch.cuda.empty_cache()
    numba.select_device(0)
    numba.close()
    numba.select_device(0)
    print("GPU Usage after emptying the cache")
    gpu_usage()

In [ ]:
# 1.0 load in datasets

def load_datasets(print_out = False, place_on_gpu=True):
    # load files
    train_feats = np.load("data/train_feats.npy", allow_pickle=True)
    train_labels = np.load("data/train_labels.npy", allow_pickle=True)
    test_feats = np.load("data/test_feats.npy", allow_pickle=True)
    test_labels = np.load("data/test_labels.npy", allow_pickle=True)
    val_feats = np.load("data/val_feats.npy", allow_pickle=True)
    val_labels = np.load("data/val_labels.npy", allow_pickle=True)
    # reshape numpy arrays
    train_feats = train_feats.reshape(train_feats.shape[0]*train_feats.shape[1], train_feats.shape[2])
    train_labels = train_labels.reshape(train_labels.shape[0]*train_labels.shape[1], train_labels.shape[2])
    test_feats = test_feats.reshape(test_feats.shape[0]*test_feats.shape[1], test_feats.shape[2])
    test_labels = test_labels.reshape(test_labels.shape[0]*test_labels.shape[1], test_labels.shape[2])
    val_feats = val_feats.reshape(val_feats.shape[0]*val_feats.shape[1], val_feats.shape[2])
    val_labels = val_labels.reshape(val_labels.shape[0]*val_labels.shape[1], val_labels.shape[2])
    
    if print_out:
        print ("train feats: ", train_feats.shape, " type: ", type(train_feats))
        print ("train labels: ", train_labels.shape, " type: ", type(train_labels))

        print ("test feats: ", test_feats.shape, " type: ", type(test_feats))
        print ("test labels: ", test_labels.shape, " type: ", type(test_labels))

        print ("val feats: ", val_feats.shape, " type: ", type(val_feats))
        print ("val labels: ", val_labels.shape, " type: ", type(val_labels))

    # create tensors
    train_feats_tensor = torch.tensor(train_feats, requires_grad=True, dtype=torch.float)
    train_labels_tensor = torch.tensor(train_labels, dtype=torch.float)

    test_feats_tensor = torch.tensor(test_feats, requires_grad=True, dtype=torch.float)
    test_labels_tensor = torch.tensor(test_labels, dtype=torch.float)

    val_feats_tensor = torch.tensor(val_feats, requires_grad=True, dtype=torch.float)
    val_labels_tensor = torch.tensor(val_labels, dtype=torch.float)

    if print_out:
        print ("train feats tensor: ", train_feats_tensor.shape, " type: ", type(train_feats_tensor))
        print ("train labels tensor: ", train_labels_tensor.shape, " type: ", type(train_labels_tensor))

        print ("test feats tensor: ", test_feats_tensor.shape, " type: ", type(test_feats_tensor))
        print ("test labels tensor: ", test_labels_tensor.shape, " type: ", type(test_labels_tensor))

        print ("val feats tensor: ", val_feats_tensor.shape, " type: ", type(val_feats_tensor))
        print ("val labels tensor: ", val_labels_tensor.shape, " type: ", type(val_labels_tensor))
    
    # place tensors on gpu
    if place_on_gpu:
        if torch.cuda.is_available():
            train_feats_tensor = train_feats_tensor.cuda()
            train_labels_tensor = train_labels_tensor.cuda()
            test_feats_tensor = test_feats_tensor.cuda()
            test_labels_tensor = test_labels_tensor.cuda()
            val_feats_tensor = val_feats_tensor.cuda()
            val_labels_tensor = val_labels_tensor.cuda()
            
        if print_out:
            print ("train_feats_tensor.device: ", train_feats_tensor.get_device())
            print ("train_labels_tensor.device: ", train_labels_tensor.get_device())
            print ("test_feats_tensor.device: ", test_feats_tensor.get_device())
            print ("test_labels_tensor.device: ", test_labels_tensor.get_device())
            print ("val_feats_tensor.device: ", val_feats_tensor.get_device())
            print ("val_labels_tensor.device: ", val_labels_tensor.get_device())
        
    return  train_feats_tensor, \
            train_labels_tensor, \
            test_feats_tensor, \
            test_labels_tensor, \
            val_feats_tensor, \
            val_labels_tensor

In [ ]:
# 1.2 set up the data loaders

# tensor tuple shape: output of load_datasets()
#   [ train_feats_tensor, train_labels_tensor,
#     test_feats_tensor, test_labels_tensor,
#     val_feats_tensor, val_labels_tensor ]
def set_up_dataloaders( batch_size, tensor_tuple):

    train_dataset = torch.utils.data.TensorDataset(tensor_tuple[0], tensor_tuple[1])
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True)

    test_dataset = torch.utils.data.TensorDataset(tensor_tuple[2], tensor_tuple[3])
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle = False)

    val_dataset = torch.utils.data.TensorDataset(tensor_tuple[4], tensor_tuple[5])
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle = False)
    
    return  train_dataset, train_loader, \
            test_dataset, test_loader, \
            val_dataset, val_loader

In [ ]:
# 2.2 bi-LSTM Model Architecture

class bi_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers=2, model_type='LSTM'):
        super(bi_LSTM, self).__init__()
        self.model_type = model_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(device=device) # hidden state
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).to(device=device) # cell state
        
        # print ("h0 device: ", h0.device)
        # print ("c0 device: ", c0.device)
        # print ("x device: ", x.device)
        
        x = x[:, None, :]
        #print ("init x shape: ", x.shape)
        out, _ = self.lstm(x, (h0, c0))
        #print ("lstm out shape: ", out.shape)
        out = self.fc(out[:, -1, :])
        #print ("linear out shape: ", out.shape)
        out = torch.sign(out)
        out = self.relu(out)
        
        return out

In [ ]:
# 3.1 helper functions for training

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            # get data
            inputs, labels = data
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)
            
            outputs = model(inputs)
            #print ("labels: ", labels)
            #print ("pred: ", outputs)
            total = labels.shape[0] * labels.shape[1]
            correct = 0
            for i, frame in enumerate(labels):
                #print (i, " frame: ", frame)
                #print (i, " outputs[i]: ", outputs[i])
                for val in torch.eq(frame, outputs[i]):
                    if val:
                        correct += 1
            
    return 100 * correct / total

def print_stats(iteration_list, accuracy_list, loss_list):
    # final accuracy plot        
    plt.plot(iteration_list, accuracy_list)
    plt.title("accuracy over time")
    plt.xlabel("iterations")
    plt.ylabel("accuracy")
    plt.show()
    
    # final loss plot        
    plt.plot(iteration_list, loss_list)
    plt.title("loss over time")
    plt.xlabel("iterations")
    plt.ylabel("loss")
    plt.show()

In [ ]:
# 3.0 Training

#free_gpu_cache()

# model parameters:
batch_size = 4
input_dim = 128
hidden_dim = 128
output_dim = 9
num_layers = 2

# training parameters
learning_rate = 0.001
num_epochs = 5

# get dataloaders
train_dataset, train_loader, \
test_dataset, test_loader, \
val_dataset, val_loader = set_up_dataloaders(batch_size, load_datasets(print_out=True, place_on_gpu=False))

# create model
model = bi_LSTM(input_dim, hidden_dim, batch_size, output_dim, num_layers, 'LSTM')
model.to(device=device)

# print ("device name: ", torch.cuda.get_device_name(0))
# print ("model.type: ", myModel.model_type)
# print ("model.device: ", next(myModel.parameters()).device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# lists for data collection
iter = 0
delta = 100
iteration_list = []
accuracy_list = []
loss_list = []

# perform epochs
startTime = time.time()
for epoch in range(num_epochs):
    print ("[ epoch: ", epoch, "]")
    for batch_index, (feats, labels) in enumerate(train_loader):
        # place data on GPU
        feats = feats.to(device=device).squeeze(1)
        labels = labels.to(device=device)
        # print ("feats shape: ", feats.shape)
        # print ("labels shape: ", labels.shape)
        # print ("labels: ", labels)
        
        # forward
        output = model(feats)
        #print ("output: ", output)
        loss = criterion(output, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
        
        # test accuracy and log stats
        if iter % delta == 0:
            acc = test_network(model, test_loader)
            iteration_list.append(iter)
            accuracy_list.append(acc)
            loss_list.append(loss)
            print(f'\t iteration: {iter}\t loss: {loss_list[len(loss_list)-1].item():.3f}\t accuracy: {accuracy_list[len(accuracy_list)-1]:.3f} %')
    
        # increase iteration
        iter += 1

print ("time elapsed: ", round((time.time() - startTime), 2), " sec")